In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kaggle-survey-2021/kaggle_survey_2021_responses.csv
/kaggle/input/kaggle-survey-2021/supplementary_data/kaggle_survey_2021_methodology.pdf
/kaggle/input/kaggle-survey-2021/supplementary_data/kaggle_survey_2021_answer_choices.pdf


In [2]:
# load dataset
data = pd.read_csv("../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv", low_memory = False)

In [3]:
# first five rows
data.head()

,Time from Start to Finish (seconds),Q1,Q2,Q3,Q4,Q5,Q6,Q7_Part_1,Q7_Part_2,Q7_Part_3,...,Q38_B_Part_3,Q38_B_Part_4,Q38_B_Part_5,Q38_B_Part_6,Q38_B_Part_7,Q38_B_Part_8,Q38_B_Part_9,Q38_B_Part_10,Q38_B_Part_11,Q38_B_OTHER
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,In which country do you currently reside?,What is the highest level of formal education ...,Select the title most similar to your current ...,For how many years have you been writing code ...,What programming languages do you use on a reg...,What programming languages do you use on a reg...,What programming languages do you use on a reg...,...,"In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor..."
1,910,50-54,Man,India,Bachelor’s degree,Other,5-10 years,Python,R,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,784,50-54,Man,Indonesia,Master’s degree,Program/Project Manager,20+ years,NaN,NaN,SQL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
3,924,22-24,Man,Pakistan,Master’s degree,Software Engineer,1-3 years,Python,NaN,NaN,...,NaN,NaN,TensorBoard,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,575,45-49,Man,Mexico,Doctoral degree,Research Scientist,20+ years,Python,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN


In [27]:
# remove the column Time from Start to Finish (seconds)
data = data.iloc[: , 1:]

In [28]:
# size of the dataset
data.shape # 25973 rows, 369 columns

(25974, 368)

In [29]:
def rename_columns(df):
    """
    input: the dataset we want to rename the columns
    output: combine the first row of the dataset into the original column
    """
    original_columns = df.columns
    num_col = df.shape[1]
    first_row = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data below the first row
    # create a list containing new column names
    new_cols = []
    for col in range(num_col):
        new_col_name = original_columns[col] + '_' + first_row[col]
        new_cols.append(new_col_name)
    df.columns = new_cols # assign the new column names to the dataset
    return df

In [30]:
def replace_nan(df):
    """
    input:
    df - the target dataset
    
    output:
    a new dataset with nan values replaced as 0 and non-nan values replaced with 1
    """
    array = np.where(df.isnull(),0,1)
    df = pd.DataFrame(data=array, columns=df.columns)
    
    return df


#role_data = replace_nan(role_question)

In [31]:
renamed_data = rename_columns(data)

In [56]:
def split_cols(df):
    """
    input:
    df - target dataframe
    
    output:
    single_questions - a list of column names that belong to single question column
    multiple_questions - a list of column names that belong to multiple question column
    """
    single_questions = []
    multiple_questions = []
    for col in df.columns:
        if 'part' in col.lower() or 'other' in col.lower():
            multiple_questions.append(col)
        else:
            single_questions.append(col)
    return single_questions, multiple_questions

single_questions = split_cols(renamed_data)[0]
multiple_questions = split_cols(renamed_data)[1]

In [41]:
test = renamed_data[[split_cols(renamed_data)[0][0]]]

In [53]:
def pivot_col(df, col):
    """
    input:
    df - target dataset
    col - the column we want to pivot its value as new columns
    
    output:
    return a pivoted dataframe where columns are value from the col of old dataframe
    """
    df['participant_id'] = df.index
    pivoted_df = df.pivot(index = 'participant_id', columns=col, values=col).reset_index().iloc[: , 1:]
    pivoted_df.index = pivoted_df.index + 1
    return pivoted_df

In [60]:
def pivot_df(df):
    """
    input:
    df - targer dataframe
    var_cols - a list of column names we want to pivot
    aggr - the column we used to group by the dataset
        
    output:
    return a dataframe where each column comes from value of each col of old dataframe
    NaN value replaced with 0 while non-NaN value replaced with 1
    """
    
    pivoted_df = []
    for col in df.columns:
        if col in single_questions: ## single question answers
            pivoted = pivot_col(df[[col]], col)
            pivoted_df.append(pivoted)
        else:
            pivoted_df.append(df[[col]])
    pivoted_merged_df = pd.concat(pivoted_df, axis=1, ignore_index=False)
    return pivoted_merged_df
pivoted_data = pivot_df(renamed_data)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [62]:
binary_data = replace_nan(pivoted_data)

In [63]:
data_matrix = binary_data.dot(np.transpose(binary_data))

In [92]:
data_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,25963,25964,25965,25966,25967,25968,25969,25970,25971,25972
0,62,11,12,18,19,19,3,25,11,13,...,16,18,20,19,23,26,4,4,12,3
1,11,48,15,14,13,12,8,9,16,16,...,12,15,12,10,9,13,5,10,11,5
2,12,15,54,14,9,14,8,11,11,14,...,14,19,9,7,9,13,7,18,7,5
3,18,14,14,49,14,16,2,15,12,11,...,10,15,17,16,6,17,7,13,8,9
4,19,13,9,14,50,14,5,11,11,14,...,18,12,14,15,11,19,5,3,11,3


In [91]:
data_matrix[[5]].sort_values(by = 5, ascending = False)

,5
5,52
151,44
2143,42
10664,42
22880,42
...,...
591,0
3835,0
21566,0
7110,0


In [65]:
def closest_participant(participant_id, participant_matrix):
    """
    input:
    participant_id - target participant
    participant_matrix - matrix where shows the similarity between each participant
    
    output - the list of participants other than the target participant, ranked by similarity
    """
    participant_list = participant_matrix[[participant_id]]
    participant_list = participant_list.sort_values(by = participant_id, ascending = False)
    
    return participant_list.index[1:]



In [66]:
closest_participant(5,data_matrix)

Int64Index([  151,  2143, 10664, 22880,  7195, 18703, 13915,  3968, 13504,
             4410,
            ...
            11550,  7907,  3392, 18516, 24901,   591,  3835, 21566,  7110,
            24745],
           dtype='int64', length=25972)

In [67]:
def compensation(df , participant):
    """
    input -
    df - target dataset
    participant - the id of the participant
    
    output -
    the yearly compensation of that participant
    """
    
    compensation = df.loc[df.index == participant]['Q25_What is your current yearly compensation (approximate $USD)?'].iloc[0]
    
    return compensation

compensation(renamed_data,5)

'30,000-39,999'

In [68]:
def similar_user_compensation(df, participant_ids):
    """
    input:
    df - target dataset
    participant_ids - a list of participant ids
    output:
    the first participant id that has non-null compensation data
    """
    for participant in participant_ids:
        if compensation(df, participant) is not None:
            return compensation(df, participant)
        else:
            pass
similar_user_compensation(renamed_data,closest_participant(5,data_matrix))

'100,000-124,999'

In [93]:
test = renamed_data.loc[renamed_data.index.isin([5,151])]

In [95]:
def samle_answers(df, user_1, user_2):
    """
    input
    df - target dataset
    user_1 - index number of user 1
    user_2 - index number of user 2
    
    output
    return the column names where answer are same between user 1 and 2
    """
    same_cols = []
    answers = df.loc[df.index.isin([user_1,user_2])]
    for col in answers.columns:
        if answers[col].iloc[0] == answers[col].iloc[1]:
            same_cols.append(col)
        else:
            pass
    return same_cols

samle_answers(renamed_data, 5, 151)

['Q7_Part_1_What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Python',
 'Q8_What programming language would you recommend an aspiring data scientist to learn first? - Selected Choice',
 "Q9_Part_11_Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  Jupyter Notebook",
 'Q12_Part_5_Which types of specialized hardware do you use on a regular basis?  (Select all that apply) - Selected Choice - None',
 'Q13_Approximately how many times have you used a TPU (tensor processing unit)?',
 'Q14_Part_1_What data visualization libraries or tools do you use on a regular basis?  (Select all that apply) - Selected Choice -  Matplotlib ',
 'Q14_Part_2_What data visualization libraries or tools do you use on a regular basis?  (Select all that apply) - Selected Choice -  Seaborn ',
 'Q16_Part_1_Which of the following machine learning frameworks do you use on a r

In [96]:
samle_answers(renamed_data, 5, 200)

['Q2_What is your gender? - Selected Choice',
 'Q4_What is the highest level of formal education that you have attained or plan to attain within the next 2 years?',
 'Q7_Part_1_What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Python',
 'Q7_Part_11_What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - MATLAB',
 'Q8_What programming language would you recommend an aspiring data scientist to learn first? - Selected Choice',
 "Q9_Part_10_Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  MATLAB ",
 "Q9_Part_11_Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice -  Jupyter Notebook",
 'Q14_Part_1_What data visualization libraries or tools do you use on a regular basis?  (Select all that apply) - Selected Choice -

In [ ]:
compensation(renamed_data,20567)

In [ ]:
renamed_data.loc[renamed_data.index == 2000].iloc[:,50:60]

In [ ]:
renamed_data.loc[renamed_data.index == 20567].iloc[:,50:60]

In [ ]:
"""
def compute_correlation(df, user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    df - dataset where is a matrix of user and their pivoted answer columns
    OUTPUT
    the correlation between the matching ratings between the two users
    '''
    answer_1 = list(df.loc[df.index == user1].iloc[0])
    answer_2 = list(df.loc[df.index == user2].iloc[0])
    
    dot_product = np.vdot(answer_1, answer_2)
    
    return dot_product #return the correlation
"""

In [ ]:
def subset_data(df, col, criteria):
    """
    input:
    df: the dataset we want to subset from
    col: target columns as the filter
    criteria: value to feed the filter
    
    output:
    a new dataset which is a subset of the original one
    """
    
    new_df = df.loc[df[col] == criteria]
    
    return new_df
"""
ds_data = subset_data(renamed_data, 
           'Q5_Select the title most similar to your current role (or most recent title if retired): - Selected Choice',
           'Data Scientist')
"""

In [ ]:
def question_columns(df, query, method = 'strict'):
    """
    input: 
    df - target dataset
    query - str, query we want to find relevant infomation in the dataset. e.g. 'Q7', or 'machine learning' 
    
    output:
    a subset of data which include the columns of the query in interest
    
    method:
    if it == strict, which means we will look for the question exactly EQUALS to the query. e.g. if we search 'age', then 'language' won't
    be taken into account in this case;
    
    if it == loose, which means we will look for the question exactly CONTAINS the query. e.g. if we search 'age', then 'language' will
    be taken into account in this case.
    """
    columns = df.columns
    question_col = []
    for col in columns:
        if method == 'strict':
            col_parts = col.lower().split() # each column name will be separated into single word tokens at first
            if query.lower() in col_parts:
                question_col.append(col)
        elif method == 'loose':
            if query.lower() in col.lower():
                question_col.append(col)
    return df[question_col]

In [ ]:
question_columns(renamed_data, query = 'compensation', method = 'strict')

## What questions we want to ask here?
1. How many types of data scientists based on activity they usually do?
2. Which skill each type emphasizes at?
3. Which platform/tools each type emphazies at?
4. Demographic segmentation of each type?
5. Profession segmentation of each type?

In [ ]:
#feature_data = role_data.iloc[: , 1:]

In [ ]:
"""
def kmeans_cluster_opt(df, init = 'k-means++', max_num_cluster = 9):
    
    input: 
    df - the dataset we want to segments into cluster
    init - the way we want to initialize the starting centroid
    max_num_cluster - the max number of cluster
    
    output:
    a visualization showing the line graph indicating the optimal number of klusters, based on inertias value
    
    num_clusters = list(range(1, max_num_cluster))
    inertias = []

    for k in num_clusters:
        model = KMeans(init=init, n_clusters=k, random_state = 42)
        model.fit(df)
        inertias.append(model.inertia_)

    
    plt.plot(num_clusters, inertias, '-o')

    plt.xlabel('number of clusters (k)')
    plt.ylabel('inertia')

    plt.show()
"""

In [ ]:
#kmeans_cluster_opt(feature_data)

In [ ]:
"""
def kmeans_predict(df, init = 'k-means++', n_clusters = 4):
    
    input:
    df - dataset we want to segment into clusters
    init - the way we want to initialize the starting centroid
    n_clusters - the number of cluster
    
    output:
    labels - return an array of predictions on the cluster label of given features
    centers - centroid values of each cluster
    model = KMeans(init=init, n_clusters = n_clusters, random_state = 42)

    model.fit(df)

    labels = model.predict(df)
    
    centers = np.array(model.cluster_centers_)
    
    return labels, centers
"""